# Substitution cipher

In this exercise, we'll explore one of the most basic problems in cryptography - cracking the code of a simple substitution cipher in which each letter of the alphabet is replaced by another. Although modern cryptography uses much more complex techniques, this illustrates some of the key statistical issues such as the use of word and letter frequencies.

In [ ]:
from collections import OrderedDict
from collections import Counter
import string
import re
import numpy as np

Define a table of letter usage frequency. We'll use this later

In [ ]:
english_letter_freq = OrderedDict({'z': 0.00074, 'q': 0.00095, 'x': 0.00150, 'j': 0.00153, 'k': 0.00772,
 'v': 0.00978, 'b': 0.01492, 'p': 0.01929, 'y': 0.01974, 'g': 0.02015,
 'f': 0.02228, 'w': 0.02360, 'm': 0.02406, 'u': 0.02758, 'c': 0.02782,
 'l': 0.04025, 'd': 0.04253, 'r': 0.05987, 'h': 0.06094, 's': 0.06327,
 'n': 0.06749, 'i': 0.06966, 'o': 0.07507, 'a': 0.08167, 't': 0.09056,
 'e': 0.12702})
english_letter_order = ''.join(english_letter_freq.keys())
print('English letters ordered from least to most common:', english_letter_order)

Read in the full text of Frankenstein. For convenience, we're going to convert all upper case letters to lower case.

In [ ]:
with open('Frankenstein.txt', 'rt') as file:  
#with open('TheTimeMachine.txt', 'rt') as file:  

    orig_text = file.read()
    
orig_text = orig_text.lower()
print(orig_text)

We now create our code, where one set of letters substitutes for another. We'll do this by taking the 26 lower case letters and scrambling them into a random order. Note that we're setting the seed for the random number generator so that we get the same results every time. To generate different codes, either change the seed or comment out the np.random.seed line.

As a simplification, we only apply the substituion cypher to the letters a-z and leave numbers, abbreviations and whitespace unchanged.

In [ ]:
intab = string.ascii_lowercase
np.random.seed(12345678)
scrambled = np.random.permutation(list(intab))
outtab = ''.join(scrambled)
trantab_forward = str.maketrans(intab, outtab)
trantab_backward = str.maketrans(outtab, intab)

print('Original letters  :', intab)
print('Encrypted letters :', outtab)

Next, we use the code to translate the original text into the secret text.

In [ ]:
encrypted_text = orig_text.translate(trantab_forward)
print(encrypted_text)

If we did everything correctly, we can reverse the process and recover the original text.

In [ ]:
print(encrypted_text.translate(trantab_backward))

As a first pass, we'll go through the text, get rid of everything that is not one of the letters a-z and count the letter frequency. The first line uses an intermediate Python feature called regular expressions to strip out the characters we don't want.

In [ ]:
encrypted_atoz = re.sub(r'[^a-z]', '', encrypted_text)

encrypted_letter_freq = Counter(list(encrypted_atoz))
print('Letter usage in encrypted text')
print(Counter(list(encrypted_atoz)), '\n')
encrypted_letter_order = ''.join(sorted(encrypted_letter_freq, key=encrypted_letter_freq.get))

print('Letters ordered by usage from least to most common')
print('English   :', english_letter_order)
print('Encrypted :', encrypted_letter_order)

Assuming that the letter usage in our text matches that in our frequency table, we should be all done. Let's apply our results and see if we recover our original text.

In [ ]:
trantab = str.maketrans(encrypted_letter_order, english_letter_order)
print(encrypted_text.translate(trantab))

Something doesn't look quite right. We've definitely made progress with cracking the substitution code and some words are recognizeable. It appears that we've at least correctly identified the most common letters, but there's still a lot of garbled text.

As a next step, we'll take advantage of another property of the English language - the frequent use of the word 'the'. In fact 'the' occurs twice as often than 'and' the second most common 3-letter word.

A key advantage of identifying the word 'the' is that it helps us with the assignment of the letter h, which can be challenging since h and r appear with nearly identical frequency (h = 6.094%, r = 5.987%).

Before we update our solution to the cipher, we should be confident that we've correctly identified 'the'. Fortunately, the last letter (e) and first letter (t) are the first and second most common letters in English text and our letter frequency analysis will almost certainly identiofy them correctly.

In [ ]:
# The letters h and r occur with nearly the same frequency and are frequently reversed. 
# We can identify h, along with t and e, by finding the most common word in the English
# language 'the'

# The letter corresponding to h in the encrypted text should be in position 18 in the
# secret_letter_order string (remember, first position is 0). If it occurs in the
# wrong position, swap with letter in position 18

encrypted_letter_order_v2 = encrypted_letter_order

most_common_word = Counter(encrypted_text.split()).most_common(1)
encrypted_t = most_common_word[0][0][0]
encrypted_h = most_common_word[0][0][1]
encrypted_e = most_common_word[0][0][2]

# Find location of t in the encrypted letter order
pos_encrypted_t = encrypted_letter_order.find(encrypted_t)
print('Location of t in encrypted letter order :', pos_encrypted_t)

# Find location of h in the encrypted letter order
pos_encrypted_h = encrypted_letter_order.find(encrypted_h)
print('Location of h in encrypted letter order :', pos_encrypted_h)

# Find location of e in the encrypted letter order
pos_encrypted_e = encrypted_letter_order.find(encrypted_e)
print('Location of e in encrypted letter order :', pos_encrypted_e)

if pos_encrypted_t != 24 or pos_encrypted_e != 25:
    print("\nMisidentified 'the'\n")
else:
    print("\nIdentified 'the' with high confidence\n")
    if pos_encrypted_h != 18:
        print("h was in wrong location\n")
        templist = list(encrypted_letter_order)
        templist[pos_encrypted_h], templist[18] = templist[18], templist[pos_encrypted_h]
        encrypted_letter_order_v2 = ''.join(templist)
    else:
        print("h was in correct location\n")

print('English     :', english_letter_order)
print('Encrypted   :', encrypted_letter_order)
print('Encrypted v2:', encrypted_letter_order_v2)

In [ ]:
trantab = str.maketrans(encrypted_letter_order_v2, english_letter_order)
print(encrypted_text.translate(trantab))

Our experience with 'the' drives home the limitations of relying purely on letter frequency. Too many of the letters have frequencies that are nearly indistiguishable. The situation is complicated by the choice of text used to derive the letter frequencies since there may be subtle changes over time.

In addition to letter frequencies and word frequencies, we can make use of the relative frequencies with which letters appear as the first letter in a word. Although this doesn't give us much progress on cracking our code, the letters z, q, x and j present an interesting case. All four of these appear with low frequencies (0.074%, 0.095%, 0.150% and 0.153% respectively) and there's a significant gap between the frequency of j and that of the k (0.772%), the next letter in order of frequency. As a result, these four letters normally occupy the first four spots but with the order often scrambled, especially for z/q and x/j. They can be distinguished though by their frequency as first letter in a word.

## Progress break

At this point, we need to take advantage of other properties of English, such as the frequency of double letters, frequency of first letters or common two-letter words. Here are some links with useful information

https://blogs.sas.com/content/iml/2014/10/03/double-letter-bigrams.html  
https://www3.nd.edu/~busiforc/handouts/cryptography/cryptography%20hints.html
https://en.wikipedia.org/wiki/Letter_frequency

In [ ]:
# Calculate the first letter frequency

encrypted_words = encrypted_text.split()
for i, word in enumerate(encrypted_words):
    encrypted_words[i] = word[0]
    
Counter(encrypted_words)


In [ ]:
print(encrypted_letter_order_v2.translate(trantab_backward))
print(english_letter_order)